In [ ]:
import xarray as xr
import numpy as np
import xesmf

In [ ]:
def columnintegration(var, field, unit, hyai, hybi, ps, hyam, hybm, p0=1, limit='') :
#--------------------    
    '''
    PARAMETERS
    ----------
	var : original name of the (unintegrated) species [e.g., ozone]
   	field
        hyai
        hybi
        p0
        ps
	ta
	hyam
	hybm
        limit [optional] : can be above100hPa

    RETURNS
    -------
	columnintegrated field

    MODIFICATIONS
    -------------
    (2021-04-16) possibility for concentrations (m-3 or cm-3) 
	needs temperature field to calculate density    
	(2022-01-10) added extra argument function call : var
	needed to distinguish Molar weights   
	(2022-02-28) extra optional argument : limit 
	limit : above100hPa
    (2022-07-05) added possibility to go from extinction -> [AOD]
    (2022-07-11) added possibility to integrate aerosol volume [m-3 m-3]
    '''
    Rair  = 287.058
    Mwair =  28.97    # this should be Mwair !!!!!
    Nav   =   6.022e23
    #
    nlev, nlat, nlon = field.shape
    #
    column = np.zeros( ( nlat,nlon ) )  # new  array


    if (   var == 'ozone'    ) : Mw = 47.9982
    elif ( var == 'LNO_PROD' ) : Mw = 14.0067


#   standard : whole column
    for ilev in range(nlev) :

        pressure = hyam[ilev] * p0 + hybm[ilev] * ps[:,:] # mid-level pressure
        
#         print(pressure)
#         density  = pressure / Rair / ta[ilev,:,:]         # density    

#       airmass
        dm = np.abs( ( hyai.isel(bnds=1)[ilev] - hyai.isel(bnds=0)[ilev] ) * p0 + 
                    ( hybi.isel(bnds=1)[ilev] -hybi.isel(bnds=0)[ilev] )  * ps[:,:] ) / 9.81    # dm = dp / g

#       how much does level contributes
        w = np.zeros( ( nlat, nlon ) ) 
        if ( limit == '' ) : w[:,:] = 1.
        elif ( limit == 'above100hPa' ) :
            plim = 10000.   
            w = np.zeros( ( nlat, nlon ) )
            for ilat in range(nlat) :
                for ilon in range(nlon) :
                   pa   = hyai.isel(bnds=0)[ilev]   * p0  + hybi.isel(bnds=0)[ilev] * ps[ilat,ilon] # pressure at ilev interface
                   pb   = hyai.isel(bnds=0)[ilev] * p0  + hybi.isel(bnds=0)[ilev] * ps[ilat,ilon] # pressure at ilev+1 interface
                   pmin = np.amin([pa,pb])                                  # minimum of boundary pressures
                   pmax = np.amax([pa,pb])                                  # maximum of boundary pressures
                   w[ilat,ilon] = np.amin([1.,np.amax([0., (pmax-plim) / (pmax-pmin) ])])                    
        else : exit()

#       combination
        dm = dm * w

        if   ( unit=='mol mol-1'  ) : column = column + dm * field[ilev,:,:] * Mw / Mwair        ; newunit = 'kg m-2'
        elif ( unit=='kg kg-1'    ) : column = column + dm * field[ilev,:,:]                     ; newunit = 'kg m-2'
        else : 
            print('columnintegration : Unit not recognized : '+unit)
            quit() 

    return column, newunit

In [ ]:
if snakemake.input.get('airmass', None):
    airmass = xr.open_dataset(snakemake.input.airmass[0])
else:
    airmass = None
mmr = xr.open_dataset(snakemake.input.mmr[0])
if len(mmr.time) > 361:
    mmr = mmr.isel(time=slice(180,660))
dvar = snakemake.wildcards.variable
with xr.set_options(keep_attrs=True):
    if snakemake.wildcards.model == 'CNRM-ESM2-1':
        dim = mmr.dims
        b_bnds = mmr['b_bnds'].values.ravel().reshape((dim['lev'], dim['bnds']))
        a_bnds = mmr['ap_bnds'].values.ravel().reshape((dim['lev'], dim['bnds']))
        a_bnds = xr.DataArray(a_bnds, coords={'lev':mmr.lev}, dims=['lev', 'bnds'])
        b_bnds = xr.DataArray(b_bnds, coords={'lev':mmr.lev}, dims=['lev', 'bnds'])
        mmr = mmr.assign(ap_bnds=a_bnds,b_bnds=b_bnds)
    elif snakemake.wildcards.model == 'IPSL-CM6A-LR-INCA':
        dim = mmr.dims
        mmr = mmr.cf.guess_coord_axis()
        if 'time' in mmr['b_bnds'].dims:
            b_bnds = mmr['b_bnds'].mean(dim='time').transpose()
            a_bnds = mmr['ap_bnds'].mean(dim='time').transpose()
            b = mmr['b'].mean(dim='time')
            ap=mmr['ap'].mean(dim='time')
        else:
            b_bnds = mmr['b_bnds'].transpose()
            a_bnds = mmr['ap_bnds'].transpose()
            b = mmr['b']
            ap=mmr['ap']
        mmr = mmr.assign(ap_bnds=a_bnds,b_bnds=b_bnds, b = b, 
                             ap=mmr['ap'])
        mmr = mmr.cf.add_bounds('lev')
        mmr['lev_bounds'] = mmr['lev_bounds'].transpose().swap_dims(bounds='bnds')
        mmr['lev_bounds'].attrs['formula'] = 'p = ap + b*ps'

In [ ]:
def calc_load(mmr, airmass=None):
    variable = mmr.attrs['variable_id']
    if airmass:
        with xr.set_options(keep_attrs=True):
            output = mmr[variable]*airmass['airmass']
            
        output = output.sum(dim=mmr.cf['Z'].name)
        unit='kg m-2'
    else:
        

        formula = mmr.cf.get_bounds('lev').formula
        output = xr.DataArray(np.zeros((mmr.dims['time'],mmr.dims['lat'],mmr.dims['lon'])),
                              coords={'time':mmr.time,'lat': mmr.lat,'lon':mmr.lon},
                              dims=['time','lat','lon'])
        if formula == 'p(n,k+1/2,j,i) = ap(k+1/2) + b(k+1/2)*ps(n,j,i)':
            a = 'ap'
            p0 = 1
            a_bnds = 'ap_bnds'
        elif formula == 'p = a*p0 + b*ps':
            a = 'a'
            a_bnds = 'a_bnds'
        elif formula == 'p = ap + b*ps':
            a = 'ap'
            p0 = 1
            a_bnds = 'ap_bnds'
        for i in range(len(mmr.time)):
            if 'time' in mmr[a].dims:
                hyam=mmr[a][i]
                hybi=mmr['b_bnds'][i]
                hybm=mmr['b'][i]
                hyai=mmr[a_bnds][i]
                p0=mmr['p0'][i]
            else:
                hyam=mmr[a]
                hybi=mmr['b_bnds']
                hybm=mmr['b']
                hyai=mmr[a_bnds]
                if 'p0' in mmr.data_vars:

                    p0=mmr['p0']
                else:
                    p0=p0

            output[i,:,:], unit = columnintegration(variable,mmr[variable][i,:,:,:],
                                                    mmr[variable].units,p0=p0,
                                                    ps=mmr['ps'][i,:,:],hyam=hyam, 
                                                    hybm=hybm,hyai=hyai, 
                                                    hybi=hybi)
    return output, unit

In [ ]:
meta_dict = {
    'concdust': {
        'long_name': 'Interated dust column load',
        'standard_name' : 'integrated_load_of_dust'
    },
    
    'concso4':{
        'long_name': 'Interated SO4 column load',
        'standard_name' : 'integrated_load_of_s04'
    },
    'concpm1':{
        'long_name': 'Interated PM1 column load',
        'standard_name' : 'integrated_load_of_pm1'
    },
    'concpm2p5': {
        'long_name': 'Interated PM2.5 column load',
        'standard_name' : 'integrated_load_of_pm2p5'
    },
    'concss': {
        'long_name' : 'Integrated Sea Salt column load',
        'standard_name' : 'integrated_load_of_ss'
    },
    'conch2oaer':{
        'long_name' : 'Integrated Aerosol Water Mass',
        'standard_name':'integrated_aerosol_water_mass'
    },
    'concoa':{
        'long_name' : 'Integrated Organic Aerosol Mass',
        'standard_name':'integrated_organic_aerosol_mass'
    }
        
}

In [ ]:
load, unit = calc_load(mmr.load(), airmass)
load.attrs = mmr[mmr.variable_id].attrs.copy()
load.attrs['units'] = unit
load.attrs['long_name'] = meta_dict[dvar]['long_name']
load.attrs['standard_name'] = meta_dict[dvar]['standard_name']

In [ ]:
outds= load.to_dataset(name=dvar)

In [ ]:
outds.attrs = mmr.attrs.copy()

In [ ]:
outds.attrs['variable_id'] = dvar

with xr.set_options(keep_attrs=True):
    outds=outds.resample(time='Y').mean()

In [ ]:
outds.to_netcdf(snakemake.output.outpath)